In [12]:
import requests
import time
import datetime

def get_option_candles(symbol, resolution='5min', start=None, end=None):
    # Set defaults if start and end not provided
    if end is None:
        end = int(time.time())
    if start is None:
        # default to 7 days back
        start = end - 7 * 24 * 60 * 60

    url = "https://api.india.delta.exchange/v2/history/candles"
    params = {
        "symbol": symbol,           # Example: "C-BTC-66400-010824"
        "resolution": resolution,   # e.g., "1m", "5m", "1h"
        "start": start,
        "end": end
    }

    headers = {
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if data.get("success"):
        for candle in data["result"]:
            ts = candle["time"]
            print(datetime.datetime.fromtimestamp(ts), candle)
        return data["result"]
    else:
        print("Error fetching data:", data)
        return []

# Example usage
symbol = "C-BTC-113800-020825"  # Replace with a real option symbol
candles = get_option_candles(symbol, resolution='15m')


2025-08-02 17:30:00 {'close': 22.2551724138, 'high': 22.2551724138, 'low': 22.2551724138, 'open': 22.2551724138, 'time': 1754136000, 'volume': 45932}
2025-08-02 17:15:00 {'close': 20, 'high': 60, 'low': 7, 'open': 40, 'time': 1754135100, 'volume': 104375}
2025-08-02 17:00:00 {'close': 47, 'high': 143, 'low': 39.3, 'open': 94, 'time': 1754134200, 'volume': 45740}
2025-08-02 16:45:00 {'close': 90, 'high': 100, 'low': 7, 'open': 9, 'time': 1754133300, 'volume': 41379}
2025-08-02 16:30:00 {'close': 10, 'high': 33, 'low': 3, 'open': 20, 'time': 1754132400, 'volume': 15028}
2025-08-02 16:15:00 {'close': 35.5, 'high': 44.1, 'low': 19, 'open': 19, 'time': 1754131500, 'volume': 13354}
2025-08-02 16:00:00 {'close': 14, 'high': 59, 'low': 14, 'open': 47, 'time': 1754130600, 'volume': 5316}
2025-08-02 15:45:00 {'close': 69, 'high': 82.5, 'low': 36, 'open': 71, 'time': 1754129700, 'volume': 7434}
2025-08-02 15:30:00 {'close': 72, 'high': 90, 'low': 62, 'open': 90, 'time': 1754128800, 'volume': 1604

In [15]:
import requests
import time
import datetime

BASE_URL = "https://api.india.delta.exchange/v2"
RESOLUTION = '1h'
SECONDS_PER_CANDLE = 3600  # 1h resolution
MAX_CANDLES = 2000
ASSET = "BTC"  # or ETH

def get_all_option_symbols(asset):
    url = f"{BASE_URL}/products"
    res = requests.get(url).json()
    symbols = []

    if res.get("success"):
        for item in res["result"]:
            if item["contract_type"] == "option":
                sym = item["symbol"]  # like "C-BTC-66400-010824"
                if f"-{asset}-" in sym:
                    symbols.append(sym)

    print(f"Found {len(symbols)} {asset} options.")
    return symbols


def get_full_year_candles(symbol, resolution='1h'):
    now = int(time.time())
    one_year_ago = now - 365 * 24 * 60 * 60

    candles = []
    end = now

    while end > one_year_ago:
        start = max(one_year_ago, end - MAX_CANDLES * SECONDS_PER_CANDLE)
        
        url = f"{BASE_URL}/history/candles"
        params = {
            "symbol": symbol,
            "resolution": resolution,
            "start": start,
            "end": end
        }

        res = requests.get(url, params=params).json()
        if res.get("success"):
            chunk = res["result"]
            if not chunk:
                break
            candles = chunk + candles  # prepend to maintain chronological order
            end = chunk[0]['time'] - 1  # shift back
        else:
            print(f"Error fetching {symbol}: {res}")
            break

    print(f"{symbol}: Got {len(candles)} candles.")
    return candles

def main():
    symbols = get_all_option_symbols(ASSET)
    all_data = {}

    for symbol in symbols:
        try:
            candles = get_full_year_candles(symbol, resolution=RESOLUTION)
            all_data[symbol] = candles
        except Exception as e:
            print(f"Failed to fetch {symbol}: {e}")
    
    return all_data

# Run the script
data = main()


Found 0 BTC options.


In [17]:
# Generate sample strikes and expiries
from datetime import datetime, timedelta

def generate_btc_option_symbols():
    expiries = [datetime.today() + timedelta(days=7*i) for i in range(1, 8)]
    strikes = list(range(50000, 75000, 1000))

    symbols = []
    for expiry in expiries:
        date_str = expiry.strftime('%d%m%y')
        for strike in strikes:
            symbols.append(f'C-BTC-{strike}-{date_str}')
            symbols.append(f'P-BTC-{strike}-{date_str}')
    return symbols


In [19]:
generate_btc_option_symbols()

['C-BTC-50000-100825',
 'P-BTC-50000-100825',
 'C-BTC-51000-100825',
 'P-BTC-51000-100825',
 'C-BTC-52000-100825',
 'P-BTC-52000-100825',
 'C-BTC-53000-100825',
 'P-BTC-53000-100825',
 'C-BTC-54000-100825',
 'P-BTC-54000-100825',
 'C-BTC-55000-100825',
 'P-BTC-55000-100825',
 'C-BTC-56000-100825',
 'P-BTC-56000-100825',
 'C-BTC-57000-100825',
 'P-BTC-57000-100825',
 'C-BTC-58000-100825',
 'P-BTC-58000-100825',
 'C-BTC-59000-100825',
 'P-BTC-59000-100825',
 'C-BTC-60000-100825',
 'P-BTC-60000-100825',
 'C-BTC-61000-100825',
 'P-BTC-61000-100825',
 'C-BTC-62000-100825',
 'P-BTC-62000-100825',
 'C-BTC-63000-100825',
 'P-BTC-63000-100825',
 'C-BTC-64000-100825',
 'P-BTC-64000-100825',
 'C-BTC-65000-100825',
 'P-BTC-65000-100825',
 'C-BTC-66000-100825',
 'P-BTC-66000-100825',
 'C-BTC-67000-100825',
 'P-BTC-67000-100825',
 'C-BTC-68000-100825',
 'P-BTC-68000-100825',
 'C-BTC-69000-100825',
 'P-BTC-69000-100825',
 'C-BTC-70000-100825',
 'P-BTC-70000-100825',
 'C-BTC-71000-100825',
 'P-BTC-710